In [191]:
# Para llevar el registro
!pip install -q comet_ml
import comet_ml
comet_ml.init(project_name = 'FineTunning')

In [192]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D, Concatenate, PReLU
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Adadelta
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.regularizers import l2

In [193]:
# Se crea el experimento en Comet
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging = True,
    auto_histogram_gradient_logging = True,
    auto_histogram_activation_logging = True,
    log_code = True
)
experiment.set_name('Test 21')

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gurumasterahb/finetunning/277887e69b79491e944485fa2e5ec851



In [194]:
# Parámetros
i_width = int(178 * 1.2)
i_height = int(218 * 1.2)
input_shape = (i_width, i_height, 3)

parameters = {
    'num_class' : 2,
    'epochs' : 200,
    'batch_size' : 16,
    'learning_rate' : 0.001,
    'loss' : 'binary_crossentropy',
    'optimizer' : 'RMSprop'}

experiment.log_parameters(parameters)

In [195]:
# Loading images
train_dir = '/kaggle/input/data-finetunning/Norma_align/Norma_align/train'
val_dir = '/kaggle/input/data-finetunning/Norma_align/Norma_align/val'

# Image Generator
gen_train = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.15,  # % de desplazamiento horizontal
    height_shift_range = 0.15, # % de desplazamiento vertical
    brightness_range = [0.25, 1.25], # Porcentaje de brillo en el rango
    zoom_range = 0.3, # Zoom de hasta 30%
    channel_shift_range = 20, # Cada pixel de cada canal varía su intensidad en una vecindad de 20 unidades
    fill_mode = 'nearest', # Después de desplazamientos que dejan la imágen 'incompleta' se rellena con el valor de los pixeles más cercanos
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255.)
train = gen_train.flow_from_directory(
    train_dir,
    target_size = (i_width, i_height),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = parameters['batch_size'],
    shuffle = True)

gen_val = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    brightness_range = [0.25, 1.25],
    zoom_range = 0.3,
    channel_shift_range = 20,
    fill_mode = 'nearest',
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255.)
val = gen_val.flow_from_directory(
    val_dir,
    target_size = (i_width, i_height),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = parameters['batch_size'],
    shuffle = True)

Found 120 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [196]:
# Loading model
path_load = os.path.join('/kaggle/input/data-finetunning/best_val_loss.h5')
model_loaded = tf.keras.models.load_model(path_load)

model_0 = Sequential()

for layer in model_loaded.layers[:15]:
    layer.trainable = False
    model_0.add(layer)

# model_0.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.01)))
model_0.add(Dense(512, activation = 'PReLU'))
model_0.add(Dense(256, activation = 'PReLU'))
# model_0.add(Dropout(0.12))
model_0.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.05)))
model_0.add(Dropout(0.5))
model_0.add(Dense(64, activation = 'relu', kernel_regularizer = l2(0.1)))
model_0.add(Dense(1, activation = 'sigmoid'))
    
# model_0.summary()

In [197]:
# Checkpoint para el mejor modelo (val_accuracy)
filepath = 'best_val_loss.h5'
checkpoint = ModelCheckpoint(
    filepath,
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    mode = 'min')

In [198]:
# Compilación y entrenamiento
model_0.compile(
    loss = parameters['loss'],
    optimizer = RMSprop(learning_rate = parameters['learning_rate']),
    metrics = ['accuracy'])

model_0.fit(
    train,
    epochs = parameters['epochs'],
    verbose = 1,
    validation_data = val,
    callbacks = [checkpoint])

Epoch 1/200
8/8 [==============================] - ETA: 0s - loss: 15.3812 - accuracy: 0.5250
Epoch 1: val_loss improved from inf to 13.35127, saving model to best_val_loss.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 6s 446ms/step - loss: 15.3812 - accuracy: 0.5250 - val_loss: 13.3513 - val_accuracy: 0.5750
Epoch 2/200
8/8 [==============================] - ETA: 0s - loss: 12.3361 - accuracy: 0.5500
Epoch 2: val_loss improved from 13.35127 to 11.16393, saving model to best_val_loss.h5
8/8 [==============================] - 4s 447ms/step - loss: 12.3361 - accuracy: 0.5500 - val_loss: 11.1639 - val_accuracy: 0.5750
Epoch 3/200
8/8 [==============================] - ETA: 0s - loss: 10.3267 - accuracy: 0.5750
Epoch 3: val_loss improved from 11.16393 to 9.40693, saving model to best_val_loss.h5
8/8 [==============================] - 3s 445ms/step - loss: 10.3267 - accuracy: 0.5750 - val_loss: 9.4069 - val_accuracy: 0.6000
Epoch 4/200
8/8 [==============================] - ETA: 0s - loss: 8.7249 - accuracy: 0.6917
Epoch 4: val_loss improved from 9.40693 to 7.91931, saving model to best_val_loss.h5
8/8 [==============================] - 3s 443ms/step - loss: 8.7249 -

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 4s 454ms/step - loss: 1.5614 - accuracy: 0.7500 - val_loss: 1.5594 - val_accuracy: 0.5000
Epoch 15/200
8/8 [==============================] - ETA: 0s - loss: 1.3172 - accuracy: 0.8000
Epoch 15: val_loss improved from 1.55937 to 1.49562, saving model to best_val_loss.h5
8/8 [==============================] - 4s 538ms/step - loss: 1.3172 - accuracy: 0.8000 - val_loss: 1.4956 - val_accuracy: 0.5500
Epoch 16/200
8/8 [==============================] - ETA: 0s - loss: 1.2396 - accuracy: 0.7417
Epoch 16: val_loss improved from 1.49562 to 1.09223, saving model to best_val_loss.h5
8/8 [==============================] - 4s 469ms/step - loss: 1.2396 - accuracy: 0.7417 - val_loss: 1.0922 - val_accuracy: 0.8000
Epoch 17/200
8/8 [==============================] - ETA: 0s - loss: 1.0154 - accuracy: 0.7667
Epoch 17: val_loss improved from 1.09223 to 0.98523, saving model to best_val_loss.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 4s 462ms/step - loss: 1.0154 - accuracy: 0.7667 - val_loss: 0.9852 - val_accuracy: 0.7500
Epoch 18/200
8/8 [==============================] - ETA: 0s - loss: 0.9264 - accuracy: 0.7667
Epoch 18: val_loss did not improve from 0.98523
8/8 [==============================] - 3s 411ms/step - loss: 0.9264 - accuracy: 0.7667 - val_loss: 1.1439 - val_accuracy: 0.5250
Epoch 19/200
8/8 [==============================] - ETA: 0s - loss: 0.8181 - accuracy: 0.8000
Epoch 19: val_loss improved from 0.98523 to 0.81985, saving model to best_val_loss.h5
8/8 [==============================] - 3s 425ms/step - loss: 0.8181 - accuracy: 0.8000 - val_loss: 0.8198 - val_accuracy: 0.7000
Epoch 20/200
8/8 [==============================] - ETA: 0s - loss: 0.9178 - accuracy: 0.8167
Epoch 20: val_loss improved from 0.81985 to 0.77012, saving model to best_val_loss.h5
8/8 [==============================] - 3s 429ms/step - loss: 0.9178 - accuracy: 0.8167 - val_loss: 0.7701 - val

In [199]:
experiment.log_model("Facial_r", filepath)
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/finetunning/277887e69b79491e944485fa2e5ec851
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [200]                : (0.5249999761581421, 0.9166666865348816)
COMET INFO:     batch_accuracy [200]          : (0.375, 1.0)
COMET INFO:     batch_loss [200]              : (0.13878275454044342, 17.748416900634766)
COMET INFO:     epoch_duration [200]          : (2.7670555709992186, 5.790258657998493)
COMET INFO:     loss [200]                    : (0.27678218483924866, 15.381244659423828)
COMET INFO:     val_accuracy [200]            : (0.5, 0.8999999761581421)
COMET INFO:     val_loss [200]             